# My Hack: Classifying mergers in Eagle simulations

What are eagle simulations?
why eagle? 
long term goal 
short term goal 

## Classifying mergers by their Size and Mass Ratio

### Training set 

<img src="Images/EM.png" width=800 height=800 />

### Procedure

<img src="Images/classification.png" width=800 height=800 />

In [ ]:
#link to images on drive
#link to mr and sr on drive

When I read that I had to pick a picture that inspires me, this is the first thing that came to mind. I am in this picture and also not. This picture makes me curious and makes me humble. It inspires a perspective that is very acomodating and accepting. This is the only time where I have felt both insiginificant and yet very calm. It makes me feel safe and I feel like I belong. 